In [1]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import codecs
import math

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from gensim.models import KeyedVectors

In [2]:
from pylab import rcParams
rcParams['figure.figsize'] = 16, 8
%matplotlib inline
nlp = spacy.load('es')

In [3]:
doc = nlp('Las naranjas y las manzanas se parecen')
pd.DataFrame([[word.text, word.tag_, word.pos_] for word in doc], columns=['Token', 'TAG', 'POS'])

,Token,TAG,POS
0,Las,DET__Definite=Def|Gender=Fem|Number=Plur|PronT...,DET
1,naranjas,NOUN__Gender=Fem|Number=Plur,NOUN
2,y,CCONJ___,CONJ
3,las,DET__Definite=Def|Gender=Fem|Number=Plur|PronT...,DET
4,manzanas,NOUN__Gender=Fem|Number=Plur,NOUN
5,se,PRON__Person=3,PRON
6,parecen,VERB__Mood=Ind|Number=Plur|Person=3|Tense=Pres...,VERB


In [4]:
pd.DataFrame([[word.text, math.exp(word.prob), word.prob] for word in doc], columns=['Token', 'Prob', 'Log Prob'])

,Token,Prob,Log Prob
0,Las,2.061154e-09,-20.0
1,naranjas,2.061154e-09,-20.0
2,y,2.061154e-09,-20.0
3,las,2.061154e-09,-20.0
4,manzanas,2.061154e-09,-20.0
5,se,2.061154e-09,-20.0
6,parecen,2.061154e-09,-20.0


In [5]:
doc2 = nlp(u"La premier alemana Angela Merkel visitó Buenos Aires esta semana")
for ent in doc2.ents:
    print('{} \t {}'.format(ent, ent.label_))

Angela Merkel 	 PER
Buenos Aires 	 LOC


In [6]:
orange = doc[1]
apple = doc[4]
orange.similarity(apple)

0.7135406

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
from collections import Counter
from nltk.corpus import stopwords

In [8]:
url = "https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/research_paper.csv"
df = pd.read_csv(url)

In [9]:
df.isnull().sum()

Title         0
Conference    0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.33, random_state=42)
print('Research title sample:', train['Title'].iloc[0])
print('Conference of this paper:', train['Conference'].iloc[0])
print('Training Data Shape:', train.shape)
print('Testing Data Shape:', test.shape)

Research title sample: Cooperating with Smartness: Using Heterogeneous Smart Antennas in Ad-Hoc Networks.
Conference of this paper: INFOCOM
Training Data Shape: (1679, 2)
Testing Data Shape: (828, 2)


# Sample

In [1]:
filepath = 'sample_esp_fixed.txt'  
a= []
with open(filepath) as fp:  
    line = fp.readline()
    cnt = 1
    while line:
        #print("Line {}: {}".format(cnt, line.strip()))
        line = fp.readline()
        cnt += 1
        a.append(line)

In [2]:
a = [item.split('?')[0] for item in a]

In [3]:
a

['¿Cuántas patas tiene una araña',
 '¿Cuál es el río más caudaloso del mundo',
 '¿Cada cuántos años tenemos un año bisiesto',
 '¿Cuántos meses tienen 28 días',
 '¿Qué es un siglo',
 '¿Quién fue Cleopatra',
 '¿Qué planeta es el más cercano al Sol',
 '¿Cuál es la montaña más alta en la actualidad',
 '¿Que nombre tiene el sonido que hace una oveja',
 '¿En qué ciudad y en qué país se encuentra la Torre Eiffel',
 '¿Para qué sirven las arterias y las venas',
 '¿Qué es un delta',
 'Si alguien de España habla español, alguien de Portugal portugués y alguien de Francia francés. ¿Qué habla alguien de Brasil',
 '¿Qué huesos encontramos en el antebrazo',
 '¿Qué tipo de palabra es "aquí"',
 '¿Qué quiere decir que un triángulo sea isóceles',
 'Si decimos que estamos en el XIX/XI/MMXVIII, ¿de qué fecha estamos hablando',
 '¿Qué es un mamífero',
 '¿Cuáles son los cinco continentes',
 '¿Cuándo termina la Edad Antigua',
 '¿Cuáles son los estados de la materia',
 '¿A qué temperatura se congela el agua',


In [4]:
from gensim import corpora, models, similarities

In [9]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in a]

In [10]:
texts

[['cuantas', 'patas', 'tiene', 'una', 'arana'],
 ['cual', 'es', 'el', 'rio', 'mas', 'caudaloso', 'del', 'mundo'],
 ['cada', 'cuantos', 'anos', 'tenemos', 'un', 'ano', 'bisiesto'],
 ['cuantos', 'meses', 'tienen', '28', 'dias'],
 ['que', 'es', 'un', 'siglo'],
 ['quien', 'fue', 'cleopatra'],
 ['que', 'planeta', 'es', 'el', 'mas', 'cercano', 'al', 'sol'],
 ['cual', 'es', 'la', 'montana', 'mas', 'alta', 'en', 'la', 'actualidad'],
 ['que', 'nombre', 'tiene', 'el', 'sonido', 'que', 'hace', 'una', 'oveja'],
 ['en',
  'que',
  'ciudad',
  'y',
  'en',
  'que',
  'pais',
  'se',
  'encuentra',
  'la',
  'torre',
  'eiffel'],
 ['para', 'que', 'sirven', 'las', 'arterias', 'y', 'las', 'venas'],
 ['que', 'es', 'un', 'delta'],
 ['si',
  'alguien',
  'de',
  'espana',
  'habla',
  'espanol,',
  'alguien',
  'de',
  'portugal',
  'portugues',
  'y',
  'alguien',
  'de',
  'francia',
  'frances.',
  'que',
  'habla',
  'alguien',
  'de',
  'brasil'],
 ['que', 'huesos', 'encontramos', 'en', 'el', 'antebr

In [11]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [27]:
for text in texts:
    for token in text:
        print(token)

cuantas
tiene
una
cual
es
el
mas
del
mundo
cada
cuantos
un
cuantos
que
es
un
siglo
quien
que
es
el
mas
al
cual
es
la
mas
en
la
que
tiene
el
sonido
que
una
en
que
y
en
que
pais
se
la
para
que
las
y
las
que
es
un
si
alguien
de
habla
alguien
de
y
alguien
de
que
habla
alguien
de
que
en
el
que
tipo
de
palabra
es
que
quiere
decir
que
un
triangulo
sea
si
que
estamos
en
el
de
que
estamos
que
es
un
cuales
son
los
la
cuales
son
los
de
la
que
se
el
quien
es
primo
cual
es
el
personaje
mas
de
de
para
que
un
en
cuantos
una
cual
es
el
pais
con
del
mundo
cuales
son
los
cuantas
tiene
la
palabra
que
que
que
tipo
de
es
un
cual
es
la
de
la
palabra
que
son
los
como
se
las
tres
que
con
los
de
un
triangulo
si
manzanas
y
le
el
de
cuantas
manzanas
le
cuales
son
los
del
en
que
los
que
quiere
decir
que
una
palabra
sea
si
en
una
al
que
en
que
y
en
cual
cuales
son
los
tres
que
es
un
primo
cuantos
son
una
como
se
los
del
en
los
cuales
el
sonido
la
cual
es
la
de
cada
cuantos
un
siglo
que
es
el
en
un
como
se
al
perso

In [12]:
frequency

defaultdict(int,
            {'cuantas': 3,
             'patas': 1,
             'tiene': 3,
             'una': 6,
             'arana': 1,
             'cual': 7,
             'es': 15,
             'el': 11,
             'rio': 1,
             'mas': 4,
             'caudaloso': 1,
             'del': 4,
             'mundo': 2,
             'cada': 2,
             'cuantos': 5,
             'anos': 1,
             'tenemos': 1,
             'un': 11,
             'ano': 1,
             'bisiesto': 1,
             'meses': 1,
             'tienen': 1,
             '28': 1,
             'dias': 1,
             'que': 30,
             'siglo': 2,
             'quien': 3,
             'fue': 1,
             'cleopatra': 1,
             'planeta': 1,
             'cercano': 1,
             'al': 3,
             'sol': 1,
             'la': 11,
             'montana': 1,
             'alta': 1,
             'en': 12,
             'actualidad': 1,
             'nombre': 1,
             '

In [13]:
texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [14]:
texts

[['cuantas', 'tiene', 'una'],
 ['cual', 'es', 'el', 'mas', 'del', 'mundo'],
 ['cada', 'cuantos', 'un'],
 ['cuantos'],
 ['que', 'es', 'un', 'siglo'],
 ['quien'],
 ['que', 'es', 'el', 'mas', 'al'],
 ['cual', 'es', 'la', 'mas', 'en', 'la'],
 ['que', 'tiene', 'el', 'sonido', 'que', 'una'],
 ['en', 'que', 'y', 'en', 'que', 'pais', 'se', 'la'],
 ['para', 'que', 'las', 'y', 'las'],
 ['que', 'es', 'un'],
 ['si',
  'alguien',
  'de',
  'habla',
  'alguien',
  'de',
  'y',
  'alguien',
  'de',
  'que',
  'habla',
  'alguien',
  'de'],
 ['que', 'en', 'el'],
 ['que', 'tipo', 'de', 'palabra', 'es'],
 ['que', 'quiere', 'decir', 'que', 'un', 'triangulo', 'sea'],
 ['si', 'que', 'estamos', 'en', 'el', 'de', 'que', 'estamos'],
 ['que', 'es', 'un'],
 ['cuales', 'son', 'los'],
 ['la'],
 ['cuales', 'son', 'los', 'de', 'la'],
 ['que', 'se', 'el'],
 ['quien', 'es', 'primo'],
 ['cual', 'es', 'el', 'personaje', 'mas', 'de', 'de'],
 ['para', 'que', 'un'],
 ['en', 'cuantos', 'una'],
 ['cual', 'es', 'el', 'pais',

In [15]:
dictionary = corpora.Dictionary(texts)

In [16]:
print(dictionary)

Dictionary(45 unique tokens: ['cuantas', 'tiene', 'una', 'cual', 'del']...)


In [17]:
print(dictionary.token2id)

{'cuantas': 0, 'tiene': 1, 'una': 2, 'cual': 3, 'del': 4, 'el': 5, 'es': 6, 'mas': 7, 'mundo': 8, 'cada': 9, 'cuantos': 10, 'un': 11, 'que': 12, 'siglo': 13, 'quien': 14, 'al': 15, 'en': 16, 'la': 17, 'sonido': 18, 'pais': 19, 'se': 20, 'y': 21, 'las': 22, 'para': 23, 'alguien': 24, 'de': 25, 'habla': 26, 'si': 27, 'palabra': 28, 'tipo': 29, 'decir': 30, 'quiere': 31, 'sea': 32, 'triangulo': 33, 'estamos': 34, 'cuales': 35, 'los': 36, 'son': 37, 'primo': 38, 'personaje': 39, 'con': 40, 'como': 41, 'tres': 42, 'le': 43, 'manzanas': 44}


In [18]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(10, 1)],
 [(6, 1), (11, 1), (12, 1), (13, 1)],
 [(14, 1)],
 [(5, 1), (6, 1), (7, 1), (12, 1), (15, 1)],
 [(3, 1), (6, 1), (7, 1), (16, 1), (17, 2)],
 [(1, 1), (2, 1), (5, 1), (12, 2), (18, 1)],
 [(12, 2), (16, 2), (17, 1), (19, 1), (20, 1), (21, 1)],
 [(12, 1), (21, 1), (22, 2), (23, 1)],
 [(6, 1), (11, 1), (12, 1)],
 [(12, 1), (21, 1), (24, 4), (25, 4), (26, 2), (27, 1)],
 [(5, 1), (12, 1), (16, 1)],
 [(6, 1), (12, 1), (25, 1), (28, 1), (29, 1)],
 [(11, 1), (12, 2), (30, 1), (31, 1), (32, 1), (33, 1)],
 [(5, 1), (12, 2), (16, 1), (25, 1), (27, 1), (34, 2)],
 [(6, 1), (11, 1), (12, 1)],
 [(35, 1), (36, 1), (37, 1)],
 [(17, 1)],
 [(17, 1), (25, 1), (35, 1), (36, 1), (37, 1)],
 [(5, 1), (12, 1), (20, 1)],
 [(6, 1), (14, 1), (38, 1)],
 [(3, 1), (5, 1), (6, 1), (7, 1), (25, 2), (39, 1)],
 [(11, 1), (12, 1), (23, 1)],
 [(2, 1), (10, 1), (16, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1), (8

In [20]:
tfidf = models.TfidfModel(corpus)

In [21]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

In [22]:
print(tfidf[corpus[1]])

[(3, 0.3546866103990365), (4, 0.45564125616376666), (5, 0.2731485283039088), (6, 0.2171966027254392), (7, 0.45564125616376666), (8, 0.5806849550458391)]


In [23]:
vec = "patas arana"

In [24]:
new_doc = dictionary.doc2bow(vec.lower().split())

In [25]:
print(new_doc)

[]


In [ ]:
sims = index[tfidf[corpus[1]]]

In [ ]:
new = "cuantas veces quiere"

In [ ]:
new_vec = dictionary.doc2bow(new.lower().split())

## Sustantivo

In [ ]:
visited = {}
nouns = []
for word in corpus:
    if word.pos_.startswith('N') and len(word.string) < 25:
        token = word.string.strip().lower()
        if token in visited:
            visited[token] += 1
            continue
        else:
            visited[token] = 1
            nouns.append(word)
nouns = sorted(nouns, key=lambda w: -visited[w.string.strip().lower()])[:150]
df = pd.DataFrame([[w.text, visited[w.string.strip().lower()]] for w in nouns], columns=['Noun', 'Freq'])

In [ ]:
df = df[df.Freq > 1]

In [ ]:
df

### Verbos


In [ ]:
visited = {}
nouns = []
for word in corpus:
    if word.pos_.startswith('V') and len(word.string) < 25:
        token = word.string.strip().lower()
        if token in visited:
            visited[token] += 1
            continue
        else:
            visited[token] = 1
            nouns.append(word)
nouns = sorted(nouns, key=lambda w: -visited[w.string.strip().lower()])[:150]
df_V = pd.DataFrame([[w.text, visited[w.string.strip().lower()]] for w in nouns], columns=['Noun', 'Freq'])

In [ ]:
df_V = df_V[df_V.Freq > 1]
df_V

### Adjetivo

In [ ]:
visited = {}
nouns = []
for word in corpus:
    if word.pos_.startswith('ADJ') and len(word.string) < 25:
        token = word.string.strip().lower()
        if token in visited:
            visited[token] += 1
            continue
        else:
            visited[token] = 1
            nouns.append(word)
nouns = sorted(nouns, key=lambda w: -visited[w.string.strip().lower()])[:150]
df_adj = pd.DataFrame([[w.text, visited[w.string.strip().lower()]] for w in nouns], columns=['Noun', 'Freq'])

In [ ]:
df_adj

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_termcounts = vectorizer.fit_transform(a)


In [ ]:
X_train_termcounts


In [ ]:
pd.DataFrame([[word.text, word.tag_, word.pos_] for word in doc], columns=['Token', 'TAG', 'POS'])


In [ ]:
import pandas as pd
data = pd.read_csv('sample_esp.txt', header = None, error_bad_lines=False)
print(data)